In [1]:
import cv2
import numpy as np
import math
import pyautogui as p

# Initialize webcam
cap = cv2.VideoCapture(0)

# Define a dummy function for trackbar
def nothing(x):
    pass

# Create a window for the trackbars
cv2.namedWindow("Color Tracker", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Color Tracker", (300, 300))

# Create trackbars for color change
cv2.createTrackbar("Thresh", "Color Tracker", 0, 255, nothing)
cv2.createTrackbar("L_H", "Color Tracker", 0, 255, nothing)
cv2.createTrackbar("L_S", "Color Tracker", 0, 255, nothing)
cv2.createTrackbar("L_V", "Color Tracker", 0, 255, nothing)
cv2.createTrackbar("U_H", "Color Tracker", 255, 255, nothing)
cv2.createTrackbar("U_S", "Color Tracker", 255, 255, nothing)
cv2.createTrackbar("U_V", "Color Tracker", 255, 255, nothing)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (600, 500))
    cv2.rectangle(frame, (0, 1), (300, 500), (255, 0, 0), 0)
    crop_img = frame[1:500, 0:300]

    hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)

    l_h = cv2.getTrackbarPos("L_H", "Color Tracker")
    l_s = cv2.getTrackbarPos("L_S", "Color Tracker")
    l_v = cv2.getTrackbarPos("L_V", "Color Tracker")
    u_h = cv2.getTrackbarPos("U_H", "Color Tracker")
    u_s = cv2.getTrackbarPos("U_S", "Color Tracker")
    u_v = cv2.getTrackbarPos("U_V", "Color Tracker")

    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    filtered = cv2.bitwise_and(crop_img, crop_img, mask=mask)

    mask_inv = cv2.bitwise_not(mask)
    thresh_val = cv2.getTrackbarPos("Thresh", "Color Tracker")
    _, thresh = cv2.threshold(mask_inv, thresh_val, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, (3, 3), iterations=6)

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        max_contour = max(contours, key=cv2.contourArea)
        epsilon = 0.0004 * cv2.arcLength(max_contour, True)
        approx = cv2.approxPolyDP(max_contour, epsilon, True)
        hull = cv2.convexHull(max_contour)
        cv2.drawContours(crop_img, [max_contour], -1, (50, 250, 150), 2)
        cv2.drawContours(crop_img, [hull], -1, (0, 255, 0), 2)

        hull_indices = cv2.convexHull(max_contour, returnPoints=False)
        defects = cv2.convexityDefects(max_contour, hull_indices)
        defect_count = 0

        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(max_contour[s][0])
            end = tuple(max_contour[e][0])
            far = tuple(max_contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)

            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / np.pi

            if angle <= 50:
                defect_count += 1
                cv2.circle(crop_img, far, 5, [255, 255, 255], -1)

        if defect_count == 0:
            cv2.putText(frame, " ", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        elif defect_count == 1:
            p.press("space")
            cv2.putText(frame, "Play/Pause", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        elif defect_count == 2:
            p.press("up")
            cv2.putText(frame, "Volume UP", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        elif defect_count == 3:
            p.press("down")
            cv2.putText(frame, "Volume Down", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        elif defect_count == 4:
            p.press("right")
            cv2.putText(frame, "Forward", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
        else:
            pass

    except:
        pass

    cv2.imshow("Thresh", thresh)
    cv2.imshow("Filtered", filtered)
    cv2.imshow("Result", frame)

    if cv2.waitKey(25) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
